In [233]:
%reload_ext autoreload
%autoreload 2

In [234]:
import pandas as pd
import numpy as np

In [235]:
raw_data = '../data/raw'

In [236]:
trends = pd.read_csv(f'{raw_data}/googletrend.csv')
trends.head(1)

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96


In [237]:
trends['state'] = trends['file'].str.split('_', expand=True)[2]

In [238]:
trends['state'].value_counts()

SN    148
NI    148
HH    148
NW    148
BY    148
SH    148
BE    148
SL    148
HE    148
ST    148
RP    148
TH    148
BW    148
Name: state, dtype: int64

In [239]:
store_states['State'].value_counts()

NW       286
BY       180
SH       115
HE       112
BE        92
SN        75
BW        73
ST        56
RP        40
TH        36
HH        28
HB,NI     22
Name: State, dtype: int64

In [240]:
trends.loc[trends['state'] =='NI', 'state']='HB,NI'

In [241]:
trends['state'].value_counts()

SN       148
HH       148
NW       148
BY       148
SH       148
HB,NI    148
BE       148
SL       148
HE       148
ST       148
RP       148
TH       148
BW       148
Name: state, dtype: int64

In [242]:
trends['week_start'] = pd.to_datetime(trends['week'].str.split(' - ', expand=True)[0])
trends['week_start_monday'] = trends['week_start'] - pd.to_timedelta(arg=trends['week_start'].dt.weekday, unit='D')
trends['week_start_monday_str'] = trends['week_start_monday'].astype(str)

In [243]:
trends.drop(['file', 'week'], axis=1, inplace=True)

In [244]:
trends.head(1)

,trend,state,week_start,week_start_monday,week_start_monday_str
0,96,SN,2012-12-02,2012-11-26,2012-11-26


In [245]:
store_states = pd.read_csv(f'{raw_data}/store_states.csv')
store_states.head(1)

,Store,State
0,1,HE


In [246]:
train = pd.read_csv(f'{raw_data}/train.csv', low_memory=False)
train.head(1)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1


In [247]:
train = train.merge(store_states, 
                    how='left',
                    on='Store', 
                    validate='many_to_one', 
                    suffixes=(False, False))
train.head(1)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,State
0,1,5,2015-07-31,5263,555,1,1,0,1,HE


In [248]:
train[train['State'].isna()]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,State


In [249]:
train['Date'] = pd.to_datetime(train['Date'])

In [250]:
train['WeekStart_dt'] = train['Date']-pd.to_timedelta(arg=train['Date'].dt.weekday, unit='D')

In [251]:
train['WeekStart_str'] = train['WeekStart_dt'].astype(str)

In [252]:
train = train.merge(trends,
            how='left',
            left_on=['WeekStart_str', 'State'],
            right_on=['week_start_monday_str', 'state'],
            validate='many_to_many')
train.head(1)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,State,WeekStart_dt,WeekStart_str,trend,state,week_start,week_start_monday,week_start_monday_str
0,1,5,2015-07-31,5263,555,1,1,0,1,HE,2015-07-27,2015-07-27,85,HE,2015-08-02,2015-07-27,2015-07-27


In [253]:
train[train['trend'].isna()]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,State,WeekStart_dt,WeekStart_str,trend,state,week_start,week_start_monday,week_start_monday_str


In [ ]:
# TODO:
# 1. transform downloaded files into single naming convention
# 2. based on these produced denormalized (or easy to join train/test)
# 3. try to fit something and see where are we